In [1]:
import os
import pandas as pd
import datetime
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy 
import math
import h5py
import time
import copy
import sys
import feather
from sklearn.metrics import mean_absolute_error
from numpy.random import seed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
seed(123)
from tensorflow import set_random_seed
set_random_seed(456)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
view_1hot_df = feather.read_dataframe("/home/whsu014/data/view_1hot_nona_len29_float_age_impute_and_True_HG_100096inds.feather")
print(view_1hot_df.columns)
print(view_1hot_df.shape)

print(view_1hot_df['CVD_ISOSORBRIDE_DINITRATE'].unique())
view_1hot_df = view_1hot_df.drop(['CVD_ISOSORBRIDE_DINITRATE'], axis=1)
print(view_1hot_df.shape)

Index(['VSIMPLE_INDEX_MASTER', 'QUARTER', 'AGE', 'SEX', 'NZDEP', 'ETHN_1',
       'ETHN_2', 'ETHN_3', 'ETHN_4', 'ETHN_5',
       ...
       'PT_DIABETES_YR', 'PT_ATRIAL_FIBRILLATION', 'PT_IMP_FATAL_CVD',
       'TRUE_HDL', 'TRUE_LDL', 'TRUE_TRI', 'TRUE_TCL', 'TRUE_TCHDL',
       'TRUE_HBA1C', 'TRUE_EGFR'],
      dtype='object', length=191)
(2902784, 191)
[0.]
(2902784, 190)


In [3]:
print("Number of inds:", len(view_1hot_df['VSIMPLE_INDEX_MASTER'].unique()))
# set to length 28
print(view_1hot_df.shape[0]/29)
view_1hot_df = copy.deepcopy(view_1hot_df[view_1hot_df['QUARTER']!=0])
print(view_1hot_df.shape[0]/28)
# Take 90000 inds for train and validation set
# remove true values
# remove QUARTER
view_1hot_df = view_1hot_df.drop(['QUARTER'], axis=1)
train_v = copy.deepcopy(view_1hot_df.iloc[:(28*90000), :-7].values)
train_true_v = copy.deepcopy(view_1hot_df.iloc[:(28*90000), -7:].values)
print(train_v.shape)
print(train_v.shape[0]/28)
print(train_true_v.shape)
print(train_true_v.shape[0]/28)

Number of inds: 100096
100096.0
100096.0
(2520000, 182)
90000.0
(2520000, 7)
90000.0


In [4]:
# set up test set
test_df = feather.read_dataframe("/home/whsu014/data/Test_set_with_TRUE_10096inds.feather")
print(test_df.columns)
print(test_df.shape)
print("Number of inds:", len(test_df['VSIMPLE_INDEX_MASTER'].unique()))
test_v = copy.deepcopy(test_df.iloc[:, :-7].values)
test_true_v = copy.deepcopy(test_df.iloc[:, -7:].values)
print("TCHDL:", test_df.columns.get_loc('TCHDL'))
print(test_v.shape)
print(test_v.shape[0]/28)
print(test_true_v.shape)
print(test_true_v.shape[0]/28)

Index(['VSIMPLE_INDEX_MASTER', 'AGE', 'SEX', 'NZDEP', 'ETHN_1', 'ETHN_2',
       'ETHN_3', 'ETHN_4', 'ETHN_5', 'TEST',
       ...
       'PT_DIABETES_YR', 'PT_ATRIAL_FIBRILLATION', 'PT_IMP_FATAL_CVD',
       'TRUE_HDL', 'TRUE_LDL', 'TRUE_TRI', 'TRUE_TCL', 'TRUE_TCHDL',
       'TRUE_HBA1C', 'TRUE_EGFR'],
      dtype='object', length=189)
(282688, 189)
Number of inds: 10096
TCHDL: 14
(282688, 182)
10096.0
(282688, 7)
10096.0


In [5]:
def get_mean_TCHDL(train_v):
    mean_tchdl = []
    num_of_inds = train_v.shape[0]//28
    for i in range(num_of_inds):
        ind_v = train_v[(i*28):((i+1)*28), :]
        ind_tchdl = np.mean(ind_v[8:28, 14])
        mean_tchdl.append(ind_tchdl)
    print(len(mean_tchdl)) 
    return np.array(mean_tchdl).T

In [6]:
def setup_xy(time_series):
    num_of_inds = time_series.shape[0]//28
    n_features = time_series.shape[1]
    #non sequential
    _x = np.empty((num_of_inds,
                   n_features+(n_features-1)*7), 
                   dtype='float')
    ###############################################
    # Construct x and y.  Flatten times series 
    # to 1 row.
    ###############################################
    for i in range(num_of_inds):
        ind_v = copy.deepcopy(time_series[(i*28):((i+1)*28), :])
        _x[i, :n_features] = copy.deepcopy(ind_v[0, :])
        for j in range(1, 8):
            start_idx = (j-1)*(n_features-1)+n_features
            end_idx = start_idx + (n_features-1)
            _x[i, start_idx:end_idx] = copy.deepcopy(ind_v[j, 1:])
    _y = get_mean_TCHDL(time_series)
    return _x, _y

In [7]:
x_train, y_train = setup_xy(train_v)
x_test, y_test = setup_xy(test_v)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

90000
10096
(90000, 1449)
(90000,)
(10096, 1449)
(10096,)


In [10]:
start_time = time.time()

num_per_fold = train_v.shape[0]//280
print("Number per fold:", num_per_fold)

end = int(num_per_fold*1)
print("End:", end)
train1_x = x_train[:end, ]
train2_x = x_train[end:, ]
train_x_rr = np.append(train2_x, train1_x, axis=0)
val_num = int(np.ceil(train_x_rr.shape[0]/10)*1)
print("val num:", val_num)
train_x = train_x_rr[:-val_num, :]
print("Train x shape:", train_x.shape)

train1_y = y_train[:end, ]
train2_y = y_train[end:, ]
train_y_rr = np.append(train2_y, train1_y, axis=0)
train_y = train_y_rr[:-val_num]
print("Train y shape:", train_y.shape)

val_x = train_x_rr[-val_num:, :]
val_y = train_y_rr[-val_num:]
print("Val x shape:", val_x.shape)
print("Val y shape:", val_y.shape)
    
start_time = time.time()
rcv = RidgeCV(alphas=[1e-6, 1e-5, 1e-4, 
                      1e-3, 1e-2, 1e-1,
                      1.0, 10.0], 
              normalize=True,
              store_cv_values=True).fit(train_x, train_y)
print("Time to find L2 parameter through cross validation and fit model", time.time() - start_time)
print("Prediction using RidgeCV")
a = rcv.alpha_
print("Alpha:", a)
print("R^2 train:", rcv.score(train_x, train_y))
print("R^2 val:", rcv.score(val_x, val_y))

########################
# on test set
########################
print("Test x shape:", x_test.shape)
print("Test y shape:", y_test.shape)
test_yhat = rcv.predict(x_test)
print("Number of coefficients (weight vector):", len(rcv.coef_))
print("Intercept", rcv.intercept_)

Number per fold: 9000
End: 9000
val num: 9000
Train x shape: (81000, 1449)
Train y shape: (81000,)
Val x shape: (9000, 1449)
Val y shape: (9000,)
Time to find L2 parameter through cross validation and fit model 23.202205896377563
Prediction using RidgeCV
Alpha: 0.01
R^2 train: 0.6618368624135442
R^2 val: 0.6573828551410142
Test x shape: (10096, 1449)
Test y shape: (10096,)
Number of coefficients (weight vector): 1449
Intercept 2.438281377512569


In [12]:
test_yhat_df = pd.DataFrame(test_yhat)
print(test_yhat_df.shape)
test_yhat_df.columns = ["MEAN_TCHDL"]
feather.write_dataframe(test_yhat_df, "/home/whsu014/data/mean_TCHDL_Ridge_Regression_yhat.feather")

(10096, 1)


In [8]:
y_test_df = pd.DataFrame(y_test)
y_test_df.columns = ["MEAN_TCHDL"]
feather.write_dataframe(y_test_df, "/home/whsu014/data/mean_TCHDL_y.feather")